## Step Four update - Aug 15th
Trying to perform svm on the dataset to see how it works

## Step Three update - Aug 6th
Perform VIF based data selection, and perform moving average to find the key event

### Step Two update - Aug 1st

After talking to my supervisor, I want to familiar myself with formal procedures of data preprocessing. So I re-organize the document into current format. 

### Step One update
I am trying to improve my machine learning skills. For that, I create this notebook to take a finished competition here: https://www.kaggle.com/c/walmart-recruiting-sales-in-stormy-weather. The data is downloaded from the competition page and re-uploaded. For a detailed information regarding this dataset, one can refer to the competition page.

This document archive my development progress on this project. More content will be added as my progress goes further. As a conclusion, all the codes will be transformed to Github. 


In [ ]:
import pandas as pd
import numpy as np
import sys
import re
df_key = pd.read_csv("../input/key.csv")
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")
df_weather = pd.read_csv("../input/weather.csv")

df_train['date'] = pd.to_datetime(df_train['date'])
df_weather['date'] = pd.to_datetime(df_weather['date'])

temp = pd.merge(df_train, df_key,how='left', on=['store_nbr'])
df_main_train = pd.merge(temp, df_weather, how='left', on=['station_nbr','date'])

print(df_train.shape)
print(temp.shape)
print(df_main_train.shape)
print(list(df_main_train))

## Weather Event Locate & Data interpolation

Highlight the data for the weather events, which is defined as rainy days with 1 inch or more rainfall, or snowy days with 2 inch or more snowfall. 

Weather event is filtered using rolling method. The dataset must be sorted in order for this method to work properly.

For data interpolation, pandas provide a convenient function: pd.interpolate()

In [ ]:
dft = df_main_train
df7 = dft.sort_values(['store_nbr','item_nbr','date']).reset_index(drop=True)

In [ ]:

df7 = df7.convert_objects(convert_numeric=True)
df7 = df7.interpolate()


patternRA = 'RA'
patternSN = 'SN'
df7['RA'], df7['SN'] = df7['codesum'].str.contains(patternRA), df7['codesum'].str.contains(patternSN)
df7['Condition'] = (df7['RA'] & (df7['preciptotal']>1.0)) | (df7['SN'] & (df7['preciptotal']>2.0))
df7['WEvent'] = (pd.rolling_mean(df7['Condition'],window=7,center=True) > 0)

mask = (df7['WEvent'] == True)
df8 = df7.loc[mask]

mask_test = (df8['item_nbr'] == 11)
df_test = df8.loc[mask_test]

## Data Selection: VIF

In [ ]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

df11 = df8

mask = (df11['item_nbr'] == 11)
df11 = df11.loc[mask]

df12 = df11[['units','tavg','depart']]
df12 = df12.convert_objects(convert_numeric=True).dropna()
df12 = df12._get_numeric_data().reset_index(drop=True)




In [ ]:
df13 = df12[['tavg','depart']]

features = "+".join(df13.columns)
y, X = dmatrices('units ~' + features, df12, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)

In [ ]:

df11 = df8

mask = (df11['item_nbr'] == 11)
df11 = df11.loc[mask]

df12 = df11[['units','snowfall','preciptotal']]
df12 = df12.convert_objects(convert_numeric=True).dropna()
df12 = df12._get_numeric_data()
df12.reset_index(drop=True)

df13 = df12[['snowfall','preciptotal']]

features = "+".join(df13.columns)
y, X = dmatrices('units ~' + features, df12, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)

In [ ]:

df11 = df8

mask = (df11['item_nbr'] == 11)
df11 = df11.loc[mask]

df12 = df11[['units','stnpressure','sealevel','resultdir','avgspeed']]
df12 = df12.convert_objects(convert_numeric=True).dropna()
df12 = df12._get_numeric_data()
df12.reset_index(drop=True)

df13 = df12[['stnpressure','sealevel','resultdir','avgspeed']]

features = "+".join(df13.columns)
y, X = dmatrices('units ~' + features, df12, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)

## Applying SVM

In [ ]:
df_test = df_test.dropna()

X = df_test[['tavg','depart','snowfall','preciptotal','stnpressure','sealevel','resultdir','avgspeed']]
y = df_test[['units']]

from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print(linreg.intercept_)
print(linreg.coef_)

y_pred = linreg.predict(X_test)
from sklearn import metrics
# MSE
print ("MSE:",metrics.mean_squared_error(y_test, y_pred))
# RMSE
print ("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

import numpy as np
import timeit
import math

# vectorized error calc
def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

#looping error calc
def rmsle_loop(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

print ("RMSLE:",rmsle(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_predict
predicted = cross_val_predict(linreg, X, y, cv=10)
# MSE
print("MSE:",metrics.mean_squared_error(y, predicted))
# RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y, predicted)))
# RMLSE
print ("RMSLE:",rmsle(y, predicted))

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(y, predicted)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
import numpy as np
import scipy as sp 
from sknn.mlp import Regressor, Layer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score 
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import datasets
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy import stats



X_trainn = X_train
X_testn = X_test

max_net = Regressor(layers= [Layer("Rectifier",units=1),
                                       Layer("Rectifier",units=1),
                                       Layer("Rectifier",units=1),
                                       Layer("Rectifier",units=1),
                                       Layer("Linear")])
params={'learning_rate': [.002],
        'hidden0__units': sp.stats.randint(2, 8),
        'hidden0__type': ["Rectifier"],
        'hidden1__units': sp.stats.randint(2, 8),
        'hidden1__type': ["Rectifier"],
        'hidden2__units': sp.stats.randint(2, 8),
        'hidden2__type': ["Rectifier"],
        'learning_rule':["adam","rmsprop","sgd"]}
max_net2 = RandomizedSearchCV(max_net,param_distributions=params,n_iter=10,cv=3,random_state=101,scoring='neg_mean_squared_error',verbose=10,\
                             pre_dispatch=None)
model_tuning=max_net2.fit(X_train,y_train)

print ("best score %s" % model_tuning.best_score_)
print ("best parameters %s" % model_tuning.best_params_)